# LLM Evaluation with MLflow

* https://mlflow.org/docs/latest/llms/llm-evaluate/notebooks/question-answering-evaluation.html

In [1]:
import os
import warnings

assert "OPENAI_API_KEY" in os.environ, "OPENAI_API_KEY environment variable must be set"
warnings.filterwarnings("ignore")

## Basic Question-Answering Evaluation

In [2]:
import mlflow
import openai
import pandas as pd

In [3]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            "What does the 'finally' block in Python do?",
            "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name="llm-qa-evaluation")

with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
    )
results.metrics

2024/11/17 12:59:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2024/11/17 12:59:09 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/17 12:59:11 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/17 12:59:11 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2024/11/17 12:59:11 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'toxicity' because it returned None.
2024/11/17 12:59:11 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2024/11/17 12:59:11 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'flesch_kinca

{'exact_match/v1': 0.0}

In [5]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,75
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The `static` keyword in a function means that ...,49
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used in conju...,62
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,65


## LLM-judged correctness with OpenAI GPT-4

In [6]:
# https://mlflow.org/docs/latest/python_api/mlflow.metrics.html#mlflow.metrics.genai.answer_similarity
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
    "learning workflows, including experiment tracking, model packaging, "
    "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
    "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "targets": "MLflow is an open-source platform for managing "
        "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
        "a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning "
        "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_similarity based on the rubric
justification: Your reasoning about the model's answer_similarity score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them be

In [7]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[
            answer_similarity_metric
        ],  # use the answer similarity metric created above
    )
results.metrics

2024/11/17 12:59:11 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/17 12:59:15 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/17 12:59:15 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2024/11/17 12:59:15 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'toxicity' because it returned None.
2024/11/17 12:59:15 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2024/11/17 12:59:15 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'flesch_kincaid_grade_level' because it returned None.
2024/11/17 12:59:15 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skippi

{'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': np.float64(3.75),
 'answer_similarity/v1/variance': np.float64(1.1875),
 'answer_similarity/v1/p90': np.float64(4.7)}

In [8]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,answer_similarity/v1/score,answer_similarity/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,72,4,The output accurately describes the `useEffect...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function indicates tha...,48,2,The output discusses the 'static' keyword in t...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used in excep...,66,5,The model's output closely aligns with the pro...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves using multiple proces...,68,4,The output aligns with the provided targets in...
